# GroupBy Mechanics
## GroupBy mechanics

In [1]:
import numpy as np
import pandas as pd

df = pd.DataFrame({'key1' : ['a', 'a', 'b', 'b', 'a'],
                   'key2' : ['one', 'two', 'one', 'two', 'one'],
                   'data1' : np.random.randn(5),
                   'data2' : np.random.randn(5)})
df

,key1,key2,data1,data2
0,a,one,0.275770,1.397019
1,a,two,0.278997,-1.141463
2,b,one,-0.288242,0.598778
3,b,two,0.388030,-0.712143
4,a,one,0.273804,-1.387421


**GroupBy on a Series using a list or array as grouping key:**

In [3]:
grouped = df['data1'].groupby(df['key1'])
grouped.mean()

key1
a    0.615600
b   -0.261302
Name: data1, dtype: float64

In [8]:
grouped.size()

key1
a    3
b    2
Name: data1, dtype: int64

In [61]:
df.loc[4, 'data1'] = np.nan
df['data1'].groupby(df['key1']).count()  # count non-NA values

key1
a    2
b    2
Name: data1, dtype: int64

In [4]:
df['data1'].groupby([df['key1'], df['key2']]).mean()

key1  key2
a     one     0.520154
      two     0.806491
b     one    -2.065765
      two     1.543160
Name: data1, dtype: float64

**GroupBy on DataFrame:**

In [6]:
# GroupBy along rows, using a column as the grouping key
df.groupby('key1').mean()
# column 'key2' is automatically excluded in the result, because it is not a numeric field

,data1,data2
key1,,
a,0.615600,-0.501074
b,-0.261302,-0.868282


In [27]:
# GroupBy on selected columns:
grouped = df.groupby('key1')[['data1']]
                         # this is a syntactic sugar for 
                         #       df[['data1']].groupby(df['key1'])
grouped.mean()

,data1
key1,
a,0.615600
b,-0.261302


In [91]:
# make the grouping key a column, instead of the index, in the resulting DataFrame:
grouped = df.groupby('key1', as_index=False)
grouped[['data1']].mean()

,key1,data1
0,a,0.258138
1,b,0.017882


In [23]:
# GroupBy along columns, using an array as the grouping key
df.groupby(df.dtypes, axis=1).size()

float64    2
object     2
dtype: int64

In [52]:
# group by index:
df1 = df.set_index('key1')

df1.groupby(level=0).mean()  # "level=0" means the 0th level of the index
df1.groupby(df1.index).mean()  # same

,data1,data2
key1,,
a,-0.185673,0.018167
b,0.017882,0.116764


## Iterating over groups

In [14]:
for name, group in df.groupby('key1'):
    print('key1 = {}:'.format(name))
    print(group)  # `group` is a DataFrame
    print('------------------------')

key1 = a:
  key1 key2     data1     data2
0    a  one  1.506997 -0.055830
1    a  two  0.806491  1.185955
4    a  one -0.466690 -2.633346
------------------------
key1 = b:
  key1 key2     data1     data2
2    b  one -2.065765 -1.380373
3    b  two  1.543160 -0.356192
------------------------


## Grouping with Dicts and Series

In [29]:
people = pd.DataFrame(np.random.randn(5, 4),
                      columns=['a', 'b', 'c', 'd'],
                      index=['Joe', 'Steve', 'Wes', 'Jim', 'Travis'])
people.iloc[2:3, [1, 2]] = np.nan
people

,a,b,c,d
Joe,-2.168504,-0.798410,0.423340,1.297117
Steve,0.379175,-1.225230,-0.306773,-1.066407
Wes,-0.577873,NaN,NaN,0.112870
Jim,1.377125,0.174179,-0.504878,-0.359892
Travis,0.605560,1.384304,0.815727,-0.184447


In [36]:
mapping_dict = {'a': 'red', 'b': 'red', 'c': 'blue', 'd': 'blue', 'e': 'red'}
mapping_series = pd.Series(mapping_dict)

mapping_series

a     red
b     red
c    blue
d    blue
e     red
dtype: object

In [37]:
people.groupby(mapping_dict, axis=1).sum()
people.groupby(mapping_series, axis=1).sum()
                        # these are the same as grouping by the array 
                        #       people.columns.map(lambda x: mapping[x])

,blue,red
Joe,1.720457,-2.966913
Steve,-1.373179,-0.846055
Wes,0.112870,-0.577873
Jim,-0.864769,1.551304
Travis,0.631279,1.989864


## Grouping with functions

In [39]:
people

,a,b,c,d
Joe,-2.168504,-0.798410,0.423340,1.297117
Steve,0.379175,-1.225230,-0.306773,-1.066407
Wes,-0.577873,NaN,NaN,0.112870
Jim,1.377125,0.174179,-0.504878,-0.359892
Travis,0.605560,1.384304,0.815727,-0.184447


In [40]:
people.groupby(len).sum()  # applies len() to the row index to get the grouping key

,a,b,c,d
3,-1.369252,-0.624230,-0.081538,1.050095
5,0.379175,-1.225230,-0.306773,-1.066407
6,0.605560,1.384304,0.815727,-0.184447


## Grouping by index levels

In [53]:
columns = pd.MultiIndex.from_arrays([['US', 'US', 'US', 'JP', 'JP'],
                                     [1, 3, 5, 1, 3]],
                                    names=['cty', 'tenor'])
hier_df = pd.DataFrame(np.random.randn(4, 5), columns=columns)
hier_df

cty          US                            JP          
tenor         1         3         5         1         3
0     -0.467945  1.574380 -0.244027  0.619579  1.144275
1      0.039812  0.244726  0.619799 -0.594026 -1.236814
2     -0.900454 -0.002005  0.274173 -0.119809  0.218871
3      1.270781  1.005243 -0.191823 -0.080954 -0.990552

In [55]:
hier_df.groupby(level=0, axis=1).count()

cty,JP,US
0,2,3
1,2,3
2,2,3
3,2,3


# Data Aggregation

## Aggregation functions

* *Aggregation* refers to transformation of array into scalar.

* Some methods have optimized implementation for GroupBy:
    * `count`: number of non-NA values
    * `sum`, `mean`
    * `median`
    * `std`, `var`
    * `min`, `max`
    * `prod`: product of non-NA values
    * `first`, `last`
    
* We can also use other functions for aggregation.

In [62]:
df

,key1,key2,data1,data2
0,a,one,-0.406134,-0.530246
1,a,two,0.922411,1.700328
2,b,one,1.051079,1.293561
3,b,two,-1.015316,-1.060032
4,a,one,NaN,-1.115581


In [63]:
df.groupby('key1').quantile(0.9)

0.9,data1,data2
key1,,
a,0.789557,1.254213
b,0.844439,1.058202


In [65]:
df.groupby('key1').agg('mean')
                # same as 
                #    df.groupby('key1').mean()
                # (valid only for the functions implemented for GroupBy)

,data1,data2
key1,,
a,0.258138,0.018167
b,0.017882,0.116764


In [70]:
def peak_to_peak(arr):
    return arr.max() - arr.min()
df.groupby('key1').agg(peak_to_peak)

,data1,data2
key1,,
a,1.328546,2.815908
b,2.066394,2.353594


## Column-wise and multiple function application

In [76]:
# multiple aggregation functions:
df.groupby('key1')['data1'] \
    .agg(['mean', 'std', ('p-2-p', peak_to_peak)])  # the 3rd function is renamed

,mean,std,p-2-p
key1,,,
a,0.258138,0.939424,1.328546
b,0.017882,1.461161,2.066394


In [82]:
# different aggregation for different columns:
grouped = df.groupby('key1')
df2 = grouped.agg({'data1': ['sum'], 'data2': ['max']})
df2

,data1,data2
,sum,max
key1,,
a,0.516277,1.700328
b,0.035763,1.293561


# General Split-Apply-Combine
## apply()

In [111]:
n = 10
students = list(range(101, 101 + n))
subjects = ['math', 'physics']
index = pd.MultiIndex.from_product([students, subjects], names=['student', 'subject'])
df = pd.DataFrame({'score': np.random.randint(10, 100, n*2)},
                  index=index)
df = df.reset_index()
df.head()

,student,subject,score
0,101,math,18
1,101,physics,50
2,102,math,13
3,102,physics,32
4,103,math,66


In [119]:
def top(df, n):
    """
    Takes a DataFrame (as 1st argument).
    Returns a DataFrame.
    """
    return df.nlargest(n, 'score')

df.groupby('subject').apply(top, n=3)
                                #^^^ other arguments to be passed to top()

student  subject  score
subject                            
math    6       104     math     76
        4       103     math     66
        10      106     math     43
physics 13      107  physics     98
        11      106  physics     87
        9       105  physics     68

In [123]:
# leave "subject" as an column, rather than making it the 1st level of index
df.groupby('subject', group_keys=False).apply(top, n=3)

,student,subject,score
6,104,math,76
4,103,math,66
10,106,math,43
13,107,physics,98
11,106,physics,87
9,105,physics,68


In [122]:
df.groupby('subject')['score'].describe()
                  # this is similar to 
                  #    df.groupby('subject')['score'].apply(lambda x: x.describe())

,count,mean,std,min,25%,50%,75%,max
subject,,,,,,,,
math,10.0,35.2,21.678458,13.0,18.5,27.5,43.0,76.0
physics,10.0,55.7,25.906027,15.0,36.5,56.0,67.5,98.0


# Pivot Tables and Cross-Tabulation
## Pivot tables

In [14]:
n = 50
np.random.seed(999)
df = pd.DataFrame({'class': np.random.choice(['A', 'B', 'C'], n),
                   'day': np.random.choice(['Mo', 'Tu', 'Wd', 'Th', 'Fr'], n),
                   'place': np.random.choice(['North', 'South'], n),
                   'value1': np.random.randint(0, 100, n),
                   'value2': np.random.randint(0, 100, n)
                  })
df.head()

,class,day,place,value1,value2
0,A,Th,South,15,51
1,A,Th,South,72,83
2,B,Mo,North,41,17
3,B,Fr,North,44,86
4,A,Tu,South,90,3


In [15]:
df.pivot_table(['value1', 'value2'], index=['class', 'day'])  
                             # computes group means by default

value1     value2
class day                      
A     Fr   43.400000  69.400000
      Mo   79.000000  33.333333
      Th   62.500000  52.250000
      Tu   67.250000  59.250000
      Wd   11.000000  47.000000
B     Fr   47.400000  49.800000
      Mo   62.500000  54.000000
      Th   62.166667  49.000000
      Tu   29.000000  50.000000
      Wd   42.500000  41.500000
C     Fr   25.500000  67.500000
      Mo   78.000000  32.500000
      Th   22.500000  33.250000
      Tu   48.000000  64.000000
      Wd   53.500000  34.500000

In [16]:
df.pivot_table(index=['class', 'day'], columns='place')

value1            value2           
place      North      South  North      South
class day                                    
A     Fr   38.50  63.000000  69.50  69.000000
      Mo   84.00  69.000000  48.50   3.000000
      Th   81.00  56.333333  36.00  57.666667
      Tu   74.00  65.000000  98.00  46.333333
      Wd   11.00        NaN  47.00        NaN
B     Fr   36.25  92.000000  43.25  76.000000
      Mo   41.00  84.000000  17.00  91.000000
      Th   48.00  65.000000  48.00  49.200000
      Tu     NaN  29.000000    NaN  50.000000
      Wd    4.00  55.333333  54.00  37.333333
C     Fr   21.00  27.000000  85.00  61.666667
      Mo   79.00  77.000000  25.00  40.000000
      Th   13.00  51.000000  28.00  49.000000
      Tu     NaN  48.000000    NaN  64.000000
      Wd   92.00  40.666667  13.00  41.666667

In [17]:
# add margins ("All" rows/columns)
df.pivot_table(index=['class', 'day'],  columns='place', margins=True)

value1                           value2                      
place      North      South        All      North      South        All
class day                                                              
A     Fr   38.50  63.000000  43.400000  69.500000  69.000000  69.400000
      Mo   84.00  69.000000  79.000000  48.500000   3.000000  33.333333
      Th   81.00  56.333333  62.500000  36.000000  57.666667  52.250000
      Tu   74.00  65.000000  67.250000  98.000000  46.333333  59.250000
      Wd   11.00        NaN  11.000000  47.000000        NaN  47.000000
B     Fr   36.25  92.000000  47.400000  43.250000  76.000000  49.800000
      Mo   41.00  84.000000  62.500000  17.000000  91.000000  54.000000
      Th   48.00  65.000000  62.166667  48.000000  49.200000  49.000000
      Tu     NaN  29.000000  29.000000        NaN  50.000000  50.000000
      Wd    4.00  55.333333  42.500000  54.000000  37.333333  41.500000
C     Fr   21.00  27.000000  25.500000  85.000000  61.666667  67.500000
      Mo   79.00  77.000000  78.000000  25.000000  40.000000  32.500000
      Th   13.00  51.000000  22.500000  28.000000  49.000000  33.250000
      Tu     NaN  48.000000  48.000000        NaN  64.000000  64.000000
      Wd   92.00  40.666667  53.500000  13.000000  41.666667  34.500000
All        43.50  56.107143  50.560000  47.954545  50.785714  49.540000

In [18]:
# specify aggregation function
df.pivot_table(index=['class', 'day'], columns='place',
               aggfunc='count', margins=True)

value1           value2          
place      North South All  North South All
class day                                  
A     Fr     4.0   1.0   5    4.0   1.0   5
      Mo     2.0   1.0   3    2.0   1.0   3
      Th     1.0   3.0   4    1.0   3.0   4
      Tu     1.0   3.0   4    1.0   3.0   4
      Wd     1.0   NaN   1    1.0   NaN   1
B     Fr     4.0   1.0   5    4.0   1.0   5
      Mo     1.0   1.0   2    1.0   1.0   2
      Th     1.0   5.0   6    1.0   5.0   6
      Tu     NaN   1.0   1    NaN   1.0   1
      Wd     1.0   3.0   4    1.0   3.0   4
C     Fr     1.0   3.0   4    1.0   3.0   4
      Mo     1.0   1.0   2    1.0   1.0   2
      Th     3.0   1.0   4    3.0   1.0   4
      Tu     NaN   1.0   1    NaN   1.0   1
      Wd     1.0   3.0   4    1.0   3.0   4
All         22.0  28.0  50   22.0  28.0  50

In [20]:
# specify value to replace NA's in the result
df.pivot_table(index=['class', 'day'], columns='place',
               aggfunc='count', fill_value=0)

value1       value2      
place      North South  North South
class day                          
A     Fr       4     1      4     1
      Mo       2     1      2     1
      Th       1     3      1     3
      Tu       1     3      1     3
      Wd       1     0      1     0
B     Fr       4     1      4     1
      Mo       1     1      1     1
      Th       1     5      1     5
      Tu       0     1      0     1
      Wd       1     3      1     3
C     Fr       1     3      1     3
      Mo       1     1      1     1
      Th       3     1      3     1
      Tu       0     1      0     1
      Wd       1     3      1     3

## Cross tabulation
A cross tabulation is a special case of a pivot table that compute group frequecies.

In [21]:
pd.crosstab(df['class'], df['day'], margins=True)

day,Fr,Mo,Th,Tu,Wd,All
class,,,,,,
A,5,3,4,4,1,17
B,5,2,6,1,4,18
C,4,2,4,1,4,15
All,14,7,14,6,9,50


In [22]:
pd.crosstab([df['class'], df['place']], df['day'], margins=True)

day          Fr  Mo  Th  Tu  Wd  All
class place                         
A     North   4   2   1   1   1    9
      South   1   1   3   3   0    8
B     North   4   1   1   0   1    7
      South   1   1   5   1   3   11
C     North   1   1   3   0   1    6
      South   3   1   1   1   3    9
All          14   7  14   6   9   50